In [18]:
# Cell 1: Setup

import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from dataclasses import dataclass
from typing import Dict, List, Optional
import time # To time our final lap

# --- Define Dataclasses (needed for loading pickle files) ---
@dataclass
class TrackGeometry:
    name: str; year: int; track_bounds: Dict[str, np.ndarray]; racing_lines: Dict[str, np.ndarray]
    elevation: Optional[np.ndarray] = None; sectors: Optional[Dict] = None; metadata: Optional[Dict] = None

# --- NEW: Tensor-based Normalizer ---
class TensorNormalizer:
    def __init__(self, data_tensor):
        # data_tensor is a PyTorch tensor
        self.min = torch.min(data_tensor, axis=0).values
        self.max = torch.max(data_tensor, axis=0).values

    def transform(self, data):
        return (data - self.min) / (self.max - self.min + 1e-8)

    def inverse_transform(self, data):
        # Automatically handles different numbers of columns
        num_cols = data.shape[1]
        min_vals = self.min[:num_cols]
        max_vals = self.max[:num_cols]
        return data * (max_vals - min_vals + 1e-8) + min_vals

# --- Physical Constants ---
F1_MASS = 798.0; F1_POWER = 746000.0; F1_DRAG_COEFF = 1.0; F1_DOWNFORCE_COEFF = 3.5
F1_FRICTION_COEFF = 2.5; AIR_DENSITY = 1.225; GRAVITY = 9.81; FRONTAL_AREA = 1.5

# --- Load Geometry for Boundaries ---
with open('data_cache/silverstone_2023_geometry.pkl', 'rb') as f:
    geometry = pickle.load(f)

inner_boundary_np = geometry.track_bounds['inner']
outer_boundary_np = geometry.track_bounds['outer']
outer_path = Path(outer_boundary_np)
inner_path = Path(inner_boundary_np)

print("Setup complete.")

Setup complete.


In [27]:
# Cell 2: The PINN Trainer Class

class RacingPINNTrainer:
    def __init__(self, model, s_tensor, ground_truth, scaler, track_length):
        self.model = model
        self.s_grad = s_tensor.clone().detach().requires_grad_(True)
        self.s_no_grad = s_tensor
        self.gt_normalized = ground_truth
        self.scaler = scaler
        self.track_length = track_length # Make sure this line is here!
        self.optimizer = optim.Adam(model.parameters(), lr=1e-4)
        self.history = {'data': [], 'physics': [], 'boundary': [], 'time': []}

    def compute_boundary_loss(self, predicted_xy_normalized):
        # Un-normalize for boundary check
        predicted_xy_unnormalized = self.scaler.inverse_transform(predicted_xy_normalized)
        outside = ~outer_path.contains_points(predicted_xy_unnormalized.detach().numpy())
        inside = inner_path.contains_points(predicted_xy_unnormalized.detach().numpy())
        violations = torch.tensor(outside | inside, dtype=torch.float32)
        return torch.mean(violations) * 100.0 # Penalty for being off track

    # In your RacingPINNTrainer class...

    def compute_physics_and_time_loss(self, predicted_normalized):
        # --- Un-normalize within the computation graph ---
        pred_unnormalized = self.scaler.inverse_transform(predicted_normalized)
        x, y, v = pred_unnormalized[:, 0], pred_unnormalized[:, 1], pred_unnormalized[:, 2]
        
        # --- NEW: Add ReLU to enforce non-negative speed ---
        v_ms = torch.relu(v) # Speed can't be negative
    
        # --- Lap Time Loss (Greed for Speed) ---
        time_loss = 1.0 / (torch.mean(v_ms) + 1e-8)
    
        # --- Physics Loss (Tire Grip) ---
        # ... (rest of the function is the same)
        dxd_s = torch.autograd.grad(x, self.s_grad, torch.ones_like(x), create_graph=True)[0]
        dyd_s = torch.autograd.grad(y, self.s_grad, torch.ones_like(y), create_graph=True)[0]
        
        d2xd_s2 = torch.autograd.grad(dxd_s, self.s_grad, torch.ones_like(dxd_s), create_graph=True)[0]
        d2yd_s2 = torch.autograd.grad(dyd_s, self.s_grad, torch.ones_like(dyd_s), create_graph=True)[0]
        
        kappa_num = torch.abs(dxd_s * d2yd_s2 - dyd_s * d2xd_s2)
        kappa_den = (dxd_s**2 + dyd_s**2)**(3/2)
        kappa = kappa_num / (kappa_den + 1e-8)
    
        downforce = 0.5 * F1_DOWNFORCE_COEFF * AIR_DENSITY * (v_ms**2) * FRONTAL_AREA
        max_grip = F1_FRICTION_COEFF * (F1_MASS * GRAVITY + downforce)
        required_grip = F1_MASS * (v_ms**2) * kappa
        
        physics_loss = torch.mean(torch.relu(required_grip - max_grip))
        return physics_loss, time_loss

    def train(self, epochs, pretrain_epochs, weights):
        print("--- STAGE 1: Pre-training ---")
        for epoch in range(pretrain_epochs):
            self.optimizer.zero_grad()
            loss = nn.functional.mse_loss(self.model(self.s_no_grad), self.gt_normalized)
            loss.backward()
            self.optimizer.step()
            if epoch % (pretrain_epochs // 5) == 0:
                print(f"Pre-train Epoch {epoch}, Data Loss: {loss.item():.5f}")

        print("\n--- STAGE 2: Physics-Informed Fine-tuning ---")
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            pred_normalized = self.model(self.s_grad)
            
            data_loss = nn.functional.mse_loss(pred_normalized, self.gt_normalized)
            boundary_loss = self.compute_boundary_loss(pred_normalized[:, :2])
            physics_loss, time_loss = self.compute_physics_and_time_loss(pred_normalized)

            total_loss = (weights['data'] * data_loss + 
                          weights['boundary'] * boundary_loss + 
                          weights['physics'] * physics_loss + 
                          weights['time'] * time_loss)
            
            total_loss.backward()

            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            self.optimizer.step()

            self.history['data'].append(data_loss.item()); self.history['boundary'].append(boundary_loss.item())
            self.history['physics'].append(physics_loss.item()); self.history['time'].append(time_loss.item())
            
            if epoch % (epochs // 10) == 0:
                print(f"Epoch {epoch} | Loss: {total_loss.item():.4f} [Data: {data_loss.item():.4f}, "
                      f"Boundary: {boundary_loss.item():.2f}, Physics: {physics_loss.item():.2f}, Time: {time_loss.item():.4f}]")

    def get_final_results(self):
        self.model.eval()
        with torch.no_grad():
            pred_normalized = self.model(self.s_no_grad)
            final_path = self.scaler.inverse_transform(pred_normalized)
        
        # Calculate Lap Time
        x, y, v = final_path[:, 0], final_path[:, 1], final_path[:, 2]
        segment_lengths = torch.sqrt((x.diff()**2) + (y.diff()**2))
        segment_times = segment_lengths / v[:-1]
        lap_time = torch.sum(segment_times)
        
        return final_path.numpy(), lap_time.item()



In [28]:
# Cell 3: Main Execution

# --- Model Definition ---
class RacingPINN(nn.Module):
    def __init__(self, num_hidden_layers=8, hidden_size=256):
        super().__init__()
        layers = [nn.Linear(1, hidden_size), nn.Tanh()]
        for _ in range(num_hidden_layers): layers.extend([nn.Linear(hidden_size, hidden_size), nn.Tanh()])
        layers.append(nn.Linear(hidden_size, 3))
        self.net = nn.Sequential(*layers)
    def forward(self, s): return self.net(s)

# --- Data Loading ---
def load_and_normalize_data(path='data_cache/silverstone_2023_training.pkl'):
    with open(path, 'rb') as f: data = pickle.load(f)
    x, y = data['positions'][0, :, 0], data['positions'][0, :, 1]
    speed_ms = data['speeds'][0, :] / 3.6
    
    gt_unnormalized = torch.tensor(np.vstack((x, y, speed_ms)).T, dtype=torch.float32)
    
    # Calculate track length
    track_length = torch.sum(torch.sqrt((gt_unnormalized[:,0].diff()**2) + (gt_unnormalized[:,1].diff()**2)))
    
    scaler = TensorNormalizer(gt_unnormalized)
    gt_normalized = scaler.transform(gt_unnormalized)
    
    s_tensor = torch.linspace(0, 1, len(x), dtype=torch.float32).view(-1, 1)
    return s_tensor, gt_normalized, scaler, track_length.item()

# --- Run Training ---
s_tensor, gt_normalized, scaler, track_length = load_and_normalize_data()
pinn_model = RacingPINN()
trainer = RacingPINNTrainer(pinn_model, s_tensor, gt_normalized, scaler, track_length)

# --- Loss Function Weights ---
# These are the most important knobs to tune.
weights = {'data': 1.0, 'boundary': 5.0, 'physics': 0.1, 'time': 0.05}

trainer.train(epochs=10000, pretrain_epochs=2000, weights=weights)

# --- Get and Display Final Results ---
final_path, lap_time = trainer.get_final_results()
print(f"\n\n--- OPTIMIZATION COMPLETE ---")
print(f"Predicted Optimal Lap Time: {lap_time:.3f} seconds")

# --- Final Visualization ---
plt.figure(figsize=(18, 8))
plt.suptitle(f"Silverstone Optimal Lap Analysis - Lap Time: {lap_time:.3f}s", fontsize=16)

# Plot 1: Track Layout
ax1 = plt.subplot(1, 2, 1)
ax1.plot(outer_boundary_np[:, 0], outer_boundary_np[:, 1], 'k-', label='Track Boundary')
ax1.plot(inner_boundary_np[:, 0], inner_boundary_np[:, 1], 'k-')
ax1.plot(final_path[:, 0], final_path[:, 1], 'r-', linewidth=2, label='PINN Optimal Line')
ax1.set_title('Optimal Racing Line'); ax1.set_xlabel('X (m)'); ax1.set_ylabel('Y (m)')
ax1.legend(); ax1.axis('equal'); ax1.grid(True)

# Plot 2: Speed Profile
ax2 = plt.subplot(1, 2, 2)
# Calculate distance along the path for plotting
distances = np.insert(np.cumsum(np.sqrt(np.sum(np.diff(final_path[:, :2], axis=0)**2, axis=1))), 0, 0)
ax2.plot(distances, final_path[:, 2] * 3.6, 'r-', label='PINN Speed') # Convert back to km/h for plot
ax2.set_title('Speed Profile'); ax2.set_xlabel('Distance (m)'); ax2.set_ylabel('Speed (km/h)')
ax2.legend(); ax2.grid(True)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

--- STAGE 1: Pre-training ---
Pre-train Epoch 0, Data Loss: 0.48467
Pre-train Epoch 400, Data Loss: 0.02457


KeyboardInterrupt: 